In [ ]:
from dotenv import load_dotenv
import os
import easyocr
import requests
from PIL import Image
import base64
import numpy as np
from openai import OpenAI

# .env 파일 로드
load_dotenv()

# 환경 변수에서 Hugging Face API 토큰 가져오기
api_token = os.getenv('HUGGING_FACE_API')
client = OpenAI(
  api_key=os.getenv("GPT_API"),  # this is also the default, it can be omitted
)

# EasyOCR 리더 초기화 (한글과 영어 지원)
ocr_reader = easyocr.Reader(['ko', 'en'])

# 이미지 불러오기
image_path = './testimage/test6.png'
image = Image.open(image_path).convert('RGB')

# 이미지 파일을 읽어서 base64로 변환
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# 1. EasyOCR을 사용하여 이미지 내 텍스트 추출 (PIL 이미지를 numpy 배열로 변환)
ocr_result = ocr_reader.readtext(np.array(image))
ocr_text = ' '.join([res[1] for res in ocr_result])

# Hugging Face API 호출 함수
def query(image_path):
    API_URL = "https://api-inference.huggingface.co/models/Salesforce/blip-image-captioning-large"
    headers = {"Authorization": f"Bearer {api_token}"}  # Hugging Face API 토큰
    image_data = encode_image(image_path)

    # API 요청 데이터 (이미지를 base64로 변환 후 전송)
    payload = {
        "inputs": {
            "image": image_data
        }
    }

    # API 호출
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# 결과 출력
result = query(image_path)
combined_caption = f"OCR Text: {ocr_text}\nGenerated Caption: {result}"

print(combined_caption)
